# Working with Loops
Let's kick it up another notch - we have six pilot samples, let's run our analysis on all of them!  

## Shell Variables
Assign the variables in this notebook.

In [ ]:
source bioinf_intro_config.sh
mkdir -p $TRIMMED $STAR_OUT

## A Brief journey into `for` loops
`for` loops take our use of the `$SAMPLE` variable to the next level! It is analogous to how you would teach a child to set the table: "FOR each place at the table, put a plate . . .,
At the shell you phrase it like this:

    for PERSON in Alice Bob Carol Dave Eve
    do
    put plate at PERSON's place
    put napkin at PERSON's place
    put fork at PERSON's place
    put spoon at PERSON's place
    put knife at PERSON's place
    done

Here is a real example:

In [ ]:
for SAMPLE in A B C D E F
    do
       echo "______${SAMPLE}________"
    done

The `for` loop in Bash is conceptually the same as in any other programming language, although the syntax may be different.  The `do` and `done` are essential - `do` needs to be before the "loop body" (what is going to be repeated) and `done` needs to be after it.

So let's try something almost useful:

In [ ]:
for SAMPLE in 27_MA_P_S38_L002_R1
    do
        echo "RUNNING SAMPLE: ${SAMPLE}"
    done

## Now for the real thing . . .
### Let's run fastq-mcf in a loop:

In [ ]:
for SAMPLE in 27_MA_P_S38_L002_R1
    do
        FASTQ="27_MA_P_S38_L002_R1"
        echo "---------------- TRIMMING: $FASTQ ----------------"
        fastq-mcf \
            $MYINFO/neb_e7600_adapters.fasta \
            $RAW_FASTQS/${FASTQ}_001.fastq.gz \
            -q 20 -x 0.5 \
            -o $TRIMMED/${FASTQ}_001.trim.fastq.gz
        
        echo "---------------- MAPPING: $FASTQ ----------------"
        STAR \
            --runMode alignReads \
            --twopassMode None \
            --genomeDir $GENOME_DIR \
            --readFilesIn $TRIMMED/${FASTQ}_001.trim.fastq.gz \
            --readFilesCommand gunzip -c \
            --outFileNamePrefix ${STAR_OUT}/${FASTQ}_ \
            --quantMode GeneCounts
    done

### And let's check the result

In [ ]:
ls ${STAR_OUT}

In [ ]:
head ${STAR_OUT}/27_MA_P_S38_L002_R1_ReadsPerGene.out.tab